In [4]:
# Imports
import ffmpeg
import os
import subprocess
import re
import pandas as pd
import numpy as np
from mutagen.easyid3 import EasyID3
import shlex

In [10]:
source = "TestFile.mp3"
output = "Outputs/OutFile.mp3"
check = "TestFile2.mp3"

In [18]:
spd = 1.35 # output speed
vol = 1.03 # volume rise ratio if any
bit = 16 # BitRate: 16K
frq = 24000 # sampling stream frequency

#Temporary parameters
src, out = source, output
# commands
stg1 = f'ffmpeg -i {src} -filter:a "atempo={spd},volume={vol}" -q:a 100 {out}'
stg2 = f"ffmpeg -i {src} -ac 1 -b:a {bit}k -ar {frq} -write_xing 0 {out}"

In [19]:
stg1

'ffmpeg -i TestFile.mp3 -filter:a "atempo=1.35,volume=1.03" -q:a 100 Outputs/OutFile.mp3'

In [8]:
cmnd = shlex.split(stg1)
cmnd

['ffmpeg',
 '-i',
 'TestFile.mp3',
 '-filter:a',
 'atempo=1.5,volume=1.0',
 '-q:a',
 '100',
 'OutFile.mp3']

In [9]:
# See if they are different
stg1.split(" ")

['ffmpeg',
 '-i',
 'TestFile.mp3',
 '-filter:a',
 '"atempo=1.5,volume=1.0"',
 '-q:a',
 '100',
 'OutFile.mp3']

**Trying to execute `ffmpeg` commands from python**

Here: https://stackoverflow.com/a/67998291/5820024

In [20]:
#Stage 1
src, out = source, output
stg1 = f'ffmpeg -i {src} -filter:a "atempo={spd},volume={vol}" -q:a 100 {out}'
subprocess.call(stg1, shell=True)

0

In [23]:
# Stage 2
src, out = out, 'Outputs/OutFileStage2.mp3'
stg2 = f"ffmpeg -i {src} -ac 1 -b:a {bit}k -ar {frq} -write_xing 0 {out}"
subprocess.call(stg2, shell=True)

0

In [24]:
stg2

'ffmpeg -i Outputs/OutFile.mp3 -ac 1 -b:a 16k -ar 24000 -write_xing 0 Outputs/OutFileStage2.mp3'

In [27]:
"ffmpeg -i Fast\023_Al_Mu_Z_minun\23AlMu_Z_minun13o14T7.9o9.3.mp3 -ac 1 -b:a 16k -ar 24000 -write_xing 0 Final\023_Al_Mu_Z_minun\23Nesr13o14T7.9o9.3.mp3"
"ffmpeg -i Outputs/OutFile.mp3 -ac 1 -b:a 16k -ar 24000 -write_xing 0 Outputs/OutFileStage2.mp3" ==stg2

True

In [ ]:
ffmpeg_command = f'ffmpeg -y -i {src} -filter:a "atempo={spd},volume={vol}" -vn -ar 44100 -ac 2 -b:a 16k -write_xing 0 {out}'